<a href="https://colab.research.google.com/github/zhaolotelli/FedLearn/blob/main/AFL_for_EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import collections
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, BatchSampler, RandomSampler

##EMNIST Dataset

In [ ]:
!pip install --quiet --upgrade tensorflow_federated
import tensorflow_federated as tff

     |████████████████████████████████| 583kB 18.1MB/s 
     |████████████████████████████████| 174kB 58.4MB/s 
     |████████████████████████████████| 706kB 54.5MB/s 
     |████████████████████████████████| 194kB 37.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

97402880/97398400 [==============================] - 1s 0us/step


In [ ]:
class Client_Data(object):
  def __init__(self, dataset, id):
    data = dataset.create_tf_dataset_for_client(id)
    pixels = np.array([example['pixels'].numpy().reshape(-1) for example in data])
    labels = np.array([example['label'].numpy() for example in data])
    self.X = pixels.astype(np.float32)
    self.y = labels.astype(np.int64)

  def __len__(self):
    return len(self.y)

## define client and server

In [ ]:
class Client(object):
  def __init__(self, id, client_data):
    self.id = id
    self.client_data = client_data

  def create_model(self, Learner, initial_params, learning_rate):
    self.model = Learner(initial_params, learning_rate)

  def update_model(self, params):
    self.model.assign_params(params)

  def train(self, epoch, batch_size):
    self.model.train(self.client_data, epoch, batch_size)
    loss = self.model.solve_loss(self.client_data)
    num_example = len(self.client_data)
    return num_example, loss

  def sgd(self, batch_size):
    loss, grads = self.model.sgd(self.client_data, batch_size)
    num_example = len(self.client_data)
    return num_example, loss, grads

In [ ]:
class Server(object):
  def __init__(self, train_data, ids, Learner, initial_params, learning_rate):
    self.ids = ids
    self.learner = Learner
    self.clients = self.set_clients(train_data)
    self.model = self.learner(initial_params, learning_rate)

  def set_clients(self, train_data):
    clients = []
    for id in self.ids:
      client_data = Client_Data(train_data, id)
      c = Client(id, client_data)
      c.create_model(self.learner, initial_params, learning_rate)
      clients.append(c)
    return clients

  def send_model(self):
    params = self.model.print_params()
    for c in self.clients:
      c.update_model(params)

  def select_client(self, select_rate):
    self.num_clients = np.maximum(1, np.int(np.floor(len(self.ids) * select_rate)))
    select_ids = np.random.choice(self.ids, self.num_clients, replace=False)
    select_clients = []
    for id in select_ids:
      loc_id = np.array([id == idx for idx in self.ids])
      ind = np.int(np.array(range(len(self.ids)))[loc_id])
      select_client = self.clients[ind]
      select_clients.append(select_client)
    return select_clients

## define training model

CNN model

In [ ]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
dev

device(type='cuda')

In [ ]:
class my_CNN(nn.Module):
  ''' 
  the CNN model structure is from https://pytorch.org/tutorials/beginner/nn_tutorial.html
  '''
  def __init__(self, initial_params, learning_rate):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size = 3, stride = 2, padding = 1)
    self.conv2 = nn.Conv2d(16, 16, kernel_size = 3, stride = 2, padding = 1)
    self.conv3 = nn.Conv2d(16, 10, kernel_size = 3, stride = 2, padding = 1)

    self.lr = learning_rate
    self.loss_func = F.cross_entropy
    if initial_params is not None:
      self.assign_params(initial_params)

  def forward(self, xb):
    xb = xb.view(-1, 1, 28, 28)
    xb = F.relu(self.conv1(xb))
    xb = F.relu(self.conv2(xb))
    xb = F.relu(self.conv3(xb))
    xb = F.avg_pool2d(xb, 4)
    return xb.view(-1, xb.size(1))

  def train(self, client_data, epoch, batch_size):
    X, y = map(torch.tensor, (client_data.X, client_data.y))
    
    if dev == torch.device("cuda"):
      # training with GPU
      X = X.to(dev)
      y = y.to(dev)
      self.to(dev)

    train_ds = TensorDataset(X, y)
    train_dl = DataLoader(train_ds, batch_size = batch_size)
    opt = optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)
    
    for _ in range(epoch):
      for xb, yb in train_dl:
        pred = self.forward(xb)
        loss = self.loss_func(pred, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()

  def sgd(self, client_data, batch_size):
    X, y = map(torch.tensor, (client_data.X, client_data.y))
    
    if dev == torch.device("cuda"):
      # training with GPU
      X = X.to(dev)
      y = y.to(dev)
      self.to(dev)

    train_ds = TensorDataset(X, y)
    train_dl = DataLoader(train_ds, 
        sampler = BatchSampler(RandomSampler(train_ds), 
        batch_size = batch_size, drop_last = False
    ))
    opt = optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)
    
    xb, yb = next(iter(train_dl))
    yb = yb.view(-1)
    pred = self.forward(xb)
    loss = self.loss_func(pred, yb)
    loss.backward()
    
    if dev == torch.device("cuda"):
      grads = [p.grad.view(-1).cpu().detach().numpy() for p in self.parameters()]
    else:
      grads = [p.grad.view(-1).detach().numpy() for p in self.parameters()]

    loss_value = self.solve_loss(client_data)

    return loss_value, grads

  def assign_params(self, params):
    self.conv1.weight = nn.Parameter(torch.tensor(params[0].reshape(16, 1, 3, 3), dtype=torch.float32))
    self.conv1.bias = nn.Parameter(torch.tensor(params[1], dtype=torch.float32))
    self.conv2.weight = nn.Parameter(torch.tensor(params[2].reshape(16, 16, 3, 3), dtype=torch.float32))
    self.conv2.bias = nn.Parameter(torch.tensor(params[3], dtype=torch.float32))
    self.conv3.weight = nn.Parameter(torch.tensor(params[4].reshape(10, 16, 3, 3), dtype=torch.float32))
    self.conv3.bias = nn.Parameter(torch.tensor(params[5], dtype=torch.float32))

  def print_params(self):
    
    if dev == torch.device("cuda"):
      params = [p.cpu().detach().numpy().reshape(-1) for p in self.parameters()]
    else:
      params = [p.detach().numpy().reshape(-1) for p in self.parameters()]
    return params
  
  def solve_loss(self, client_data):
    X = torch.tensor(client_data.X)
    y_true = torch.tensor(client_data.y)

    if dev == torch.device("cuda"):
      # compute with GPU
      X = X.to(dev)
      y_true = y_true.to(dev)

    y_pred = self.forward(X)
    return self.loss_func(y_pred, y_true).item()

  def predict_error(self, client_data):
    X = torch.tensor(client_data.X)
    y_true = client_data.y

    if dev == torch.device("cuda"):
      X = X.to(dev)
      self.to(dev)
      y_pred = F.softmax(self.forward(X), dim = 1).cpu().detach().numpy().argmax(axis = 1)
    else:
      y_pred = F.softmax(self.forward(X), dim = 1).detach().numpy().argmax(axis = 1)
    
    accuracy = sum(y_pred == y_true) / len(client_data)
    return accuracy
  

old version

In [ ]:
class my_CNN(object):
  ''' 
  the CNN model structure is from https://pytorch.org/tutorials/beginner/nn_tutorial.html
  '''
  def __init__(self, initial_params, learning_rate):
    self.conv1 = nn.Conv2d(1, 16, kernel_size = 3, stride = 2, padding = 1)
    self.conv2 = nn.Conv2d(16, 16, kernel_size = 3, stride = 2, padding = 1)
    self.conv3 = nn.Conv2d(16, 10, kernel_size = 3, stride = 2, padding = 1)
    self.lr = learning_rate
    self.loss_func = F.cross_entropy
    if initial_params is not None:
      self.assign_params(initial_params)

  def model(self, xb):
    xb = xb.view(-1, 1, 28, 28)
    xb = F.relu(self.conv1(xb))
    xb = F.relu(self.conv2(xb))
    xb = F.relu(self.conv3(xb))
    xb = F.avg_pool2d(xb, 4)
    return xb.view(-1, xb.size(1))

  def train(self, client_data, epoch, batch_size):
    X, y = map(torch.tensor, (client_data.X, client_data.y))
    
    if dev == torch.device("cuda"):
      # training with GPU
      X = X.to(dev)
      y = y.to(dev)
      self.conv1.to(dev)
      self.conv2.to(dev)
      self.conv3.to(dev)

    train_ds = TensorDataset(X, y)
    train_dl = DataLoader(train_ds, batch_size = batch_size)
    opt = optim.SGD((self.conv1.weight,
              self.conv1.bias,
              self.conv2.weight,
              self.conv2.bias,
              self.conv3.weight,
              self.conv3.bias), lr=self.lr, momentum=0.9)
    
    for _ in range(epoch):
      for xb, yb in train_dl:
        pred = self.model(xb)
        loss = self.loss_func(pred, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()

  def sgd(self, client_data, batch_size):
    X, y = map(torch.tensor, (client_data.X, client_data.y))
    
    if dev == torch.device("cuda"):
      # training with GPU
      X = X.to(dev)
      y = y.to(dev)
      self.conv1.to(dev)
      self.conv2.to(dev)
      self.conv3.to(dev)

    train_ds = TensorDataset(X, y)
    train_dl = DataLoader(train_ds, 
        sampler = BatchSampler(RandomSampler(train_ds), 
        batch_size = batch_size, drop_last = False
    ))
    opt = optim.SGD((self.conv1.weight,
              self.conv1.bias,
              self.conv2.weight,
              self.conv2.bias,
              self.conv3.weight,
              self.conv3.bias), lr=self.lr, momentum=0.9)
    
    xb, yb = next(iter(train_dl))
    yb = yb.view(-1)
    pred = self.model(xb)
    loss = self.loss_func(pred, yb)
    loss.backward()
    
    grads = []
    if dev == torch.device("cuda"):
      grads.append(self.conv1.weight.grad.view(-1).cpu().detach().numpy())
      grads.append(self.conv1.bias.grad.cpu().detach().numpy())
      grads.append(self.conv2.weight.grad.view(-1).cpu().detach().numpy())
      grads.append(self.conv2.bias.grad.cpu().detach().numpy())
      grads.append(self.conv3.weight.grad.view(-1).cpu().detach().numpy())
      grads.append(self.conv3.bias.grad.cpu().detach().numpy())
    else:
      grads.append(self.conv1.weight.grad.view(-1).detach().numpy())
      grads.append(self.conv1.bias.grad.detach().numpy())
      grads.append(self.conv2.weight.grad.view(-1).detach().numpy())
      grads.append(self.conv2.bias.grad.detach().numpy())
      grads.append(self.conv3.weight.grad.view(-1).detach().numpy())
      grads.append(self.conv3.bias.grad.detach().numpy())

    loss_value = self.solve_loss(client_data)

    return loss_value, grads

  def assign_params(self, params):
    self.conv1.weight = nn.Parameter(torch.tensor(params[0].reshape(16, 1, 3, 3), dtype=torch.float32))
    self.conv1.bias = nn.Parameter(torch.tensor(params[1], dtype=torch.float32))
    self.conv2.weight = nn.Parameter(torch.tensor(params[2].reshape(16, 16, 3, 3), dtype=torch.float32))
    self.conv2.bias = nn.Parameter(torch.tensor(params[3], dtype=torch.float32))
    self.conv3.weight = nn.Parameter(torch.tensor(params[4].reshape(10, 16, 3, 3), dtype=torch.float32))
    self.conv3.bias = nn.Parameter(torch.tensor(params[5], dtype=torch.float32))

  def print_params(self):
    
    if dev == torch.device("cuda"):
      params = [self.conv1.weight.cpu().detach().numpy().reshape(-1),
            self.conv1.bias.cpu().detach().numpy(), 
            self.conv2.weight.cpu().detach().numpy().reshape(-1),
            self.conv2.bias.cpu().detach().numpy(),
            self.conv3.weight.cpu().detach().numpy().reshape(-1),
            self.conv3.bias.cpu().detach().numpy()]
    else:
      params = [self.conv1.weight.detach().numpy().reshape(-1),
            self.conv1.bias.detach().numpy(), 
            self.conv2.weight.detach().numpy().reshape(-1),
            self.conv2.bias.detach().numpy(),
            self.conv3.weight.detach().numpy().reshape(-1),
            self.conv3.bias.detach().numpy()]
    return params
  
  def solve_loss(self, client_data):
    X = torch.tensor(client_data.X)
    y_true = torch.tensor(client_data.y)

    if dev == torch.device("cuda"):
      # compute with GPU
      X = X.to(dev)
      y_true = y_true.to(dev)

    y_pred = self.model(X)
    return self.loss_func(y_pred, y_true).item()

  def predict_error(self, client_data):
    X = torch.tensor(client_data.X)
    y_true = client_data.y

    if dev == torch.device("cuda"):
      X = X.to(dev)
      self.conv1.to(dev)
      self.conv2.to(dev)
      self.conv3.to(dev)
      y_pred = F.softmax(self.model(X), dim = 1).cpu().detach().numpy().argmax(axis = 1)
    else:
      y_pred = F.softmax(self.model(X), dim = 1).detach().numpy().argmax(axis = 1)
    
    accuracy = sum(y_pred == y_true) / len(client_data)
    return accuracy
  

## Aggregation

FedAvg

In [ ]:
class WAVGM(Server):
  def __init__(self, train_data, ids, Learner, initial_params, learning_rate):
    super(WAVGM, self).__init__(train_data, ids, Learner, initial_params, learning_rate)

  def train(self, epoch, batch_size, sub_size = None, select_rate=1):
    self.send_model()
    self.select_clients = self.select_client(select_rate)
    losses = []
    self.client_nums = []
    for client in self.select_clients:
      client_num, client_loss = client.train(epoch, batch_size)
      losses.append(client_loss)
      self.client_nums.append(client_num)
      # print('Client: {}, Local_loss: {:f}'.format(client.id, client_loss))
    self.aggregate()
    return np.sum(losses)
  
  def aggregate(self):
    total_params = [np.zeros(len(param)) for param in self.model.print_params()]
    total_num = sum(self.client_nums)
    t = 0
    for c in self.select_clients:
      for i in range(len(total_params)):
        total_params[i] = total_params[i] + self.client_nums[t] / total_num * c.model.print_params()[i]
      t += 1
    self.model.assign_params(total_params)
    return total_params

In [ ]:
def project(y):
  ''' algorithm comes from:
  https://arxiv.org/pdf/1309.1541.pdf
  '''
  u = sorted(y, reverse=True)
  x = []
  rho = 0
  for i in range(len(y)):
      if (u[i] + (1.0/(i+1)) * (1-np.sum(np.asarray(u)[:i]))) > 0:
          rho = i + 1
  lambda_ = (1.0/rho) * (1-np.sum(np.asarray(u)[:rho]))
  for i in range(len(y)):
      x.append(max(y[i]+lambda_, 0))
  return x

class AFL(Server):
  def __init__(self, train_data, ids, Learner, initial_params, learning_rate, lambda_learning_rate):
    super(AFL, self).__init__(train_data, ids, Learner, initial_params, learning_rate)
    self.lambdas = np.ones(len(self.clients)) / len(self.clients)
    self.lambda_lr = lambda_learning_rate

  def train(self, batch_size):
    self.send_model()
    losses = []
    grads = []
    for client in self.clients:
      client_num, client_loss, client_grads = client.sgd(batch_size)
      losses.append(client_loss)
      grads.append(client_grads)
      # print('Client: {}, Local_loss: {:f}'.format(client.id, client_loss))
    self.aggregate(losses, grads)
    return np.sum(losses)

  def aggregate(self, losses, grads):
    lambdas_new = self.lambdas + self.lambda_lr * np.array(losses)
    self.lambdas = project(lambdas_new)

    total_grad = [np.zeros(len(g)) for g in grads[0]]
    for lambda_, grad in zip(self.lambdas, grads):
      for i in range(len(grad)):
        total_grad[i] = total_grad[i] + grad[i] * lambda_
    
    total_params = [param for param in self.model.print_params()]
    for i in range(len(total_params)):
      total_params[i] = total_params[i] - self.model.lr * total_grad[i]
    self.model.assign_params(total_params)
    return total_params

In [ ]:
def project(y):
  ''' algorithm comes from:
  https://arxiv.org/pdf/1309.1541.pdf
  '''
  u = sorted(y, reverse=True)
  x = []
  rho = 0
  for i in range(len(y)):
      if (u[i] + (1.0/(i+1)) * (1-np.sum(np.asarray(u)[:i]))) > 0:
          rho = i + 1
  lambda_ = (1.0/rho) * (1-np.sum(np.asarray(u)[:rho]))
  for i in range(len(y)):
      x.append(max(y[i]+lambda_, 0))
  return x

class my_AFL(Server):
  def __init__(self, train_data, Learner, initial_params, param_learning_rate, lambda_learning_rate):
    super(my_AFL, self).__init__(train_data, Learner, initial_params, param_learning_rate)
    self.lambdas = np.ones(len(self.clients)) / len(self.clients)
    self.lambda_learning_rate = lambda_learning_rate
    
    self.training_rounds = 0
    self.final_params = [np.zeros(len(param)) for param in self.model.print_params()]

  def train(self, epoch, batch_size, learning_rate = None):
    self.send_model()
    losses = []
    for client in self.clients:
      if learning_rate is not None:
        client.model.lr = learning_rate
      _, client_loss = client.train(epoch, batch_size)
      losses.append(client_loss.cpu().detach().numpy())
      print('Client: {:d}, Local loss: {:f}'.format(client.id, client_loss))
    self.aggregate(losses)
    self.training_rounds += 1
    return np.sum(losses)

  def aggregate(self, losses):
    losses = np.array(losses)
    lambdas_new = self.lambdas + self.lambda_learning_rate * losses
    self.lambdas = project(lambdas_new)

    total_params = [np.zeros(len(param)) for param in self.model.print_params()]
    t = 0
    for c in self.clients:
      c_params = c.model.print_params()
      for i in range(len(total_params)):
        total_params[i] += self.lambdas[t] * c_params[i]
      t += 1
    
    tr = self.training_rounds
    for i in range(len(total_params)):
      self.final_params[i] = (tr * self.final_params[i] + total_params[i]) / (tr + 1)
    
    self.model.assign_params(total_params)
    return total_params

## Load pre-trained model params

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/MyDrive"

 AFL_EMNIST_model_saving_file0	 CIFAR10_model_saving_file5
 AFL_EMNIST_model_saving_file1	 CIFAR10_model_saving_file6
 AFL_EMNIST_model_saving_file2	 CIFAR10_model_saving_file7
 AFL_EMNIST_model_saving_file3	 CIFAR10_model_saving_file8
 AFL_EMNIST_model_saving_file4	 CIFAR10_model_saving_file9
 AFL_EMNIST_model_saving_file5	'Colab Notebooks'
 AFL_EMNIST_model_saving_file6	 EMNIST_model_saving_file0
 CIFAR10_model_accuracy.txt	 EMNIST_model_saving_file1
 CIFAR10_model_saving_file0	 EMNIST_model_saving_file2
 CIFAR10_model_saving_file1	 EMNIST_model_saving_file3
 CIFAR10_model_saving_file2	 EMNIST_model_saving_file4
 CIFAR10_model_saving_file3	 EMNIST_model_saving_file5
 CIFAR10_model_saving_file4	'Getting started.pdf'


In [ ]:
#for i, param in enumerate(final_model.print_params()):
#  param.tofile('/content/gdrive/My Drive/EMNIST_model_saving_file'+str(i))

load fedavg params

In [ ]:
read_params = [np.fromfile('/content/gdrive/My Drive/EMNIST_model_saving_file'+str(i), dtype = np.float32) for i in range(6)]

load AFL params

In [ ]:
read_params = [np.fromfile('/content/gdrive/My Drive/AFL_EMNIST_model_saving_file'+str(i), dtype = np.float32) for i in range(6)]
lambdas = np.fromfile('/content/gdrive/My Drive/AFL_EMNIST_model_saving_file'+str(6))

## Training example

training with randomly selected clients

In [ ]:
NUM_CLIENTS = 100
IDs = np.random.choice(emnist_train.client_ids, size = NUM_CLIENTS, replace=False)

training with all clients

In [ ]:
IDs = emnist_train.client_ids

training with randomly generated initial parameters

In [ ]:
initial_params = None
learning_rate = 0.1
EPOCH = 20
BATCH_SIZE = 20

training with pre-trained model parameters

In [ ]:
initial_params = read_params
learning_rate = 0.1
EPOCH = 20
BATCH_SIZE = 20

FedAvg model

In [ ]:
CNN_AVGM_fit = WAVGM(emnist_train, IDs, my_CNN, initial_params, learning_rate)

In [ ]:
ITER = 50
for i in range(ITER):
  loss = CNN_AVGM_fit.train(EPOCH, BATCH_SIZE)
  print('----------iter: {:d}/{:d}, loss: {:f}----------'.format(i+1, ITER, loss))
  if (i+1) % 5 == 0:
    final_model = CNN_AVGM_fit.model
    for k, param in enumerate(final_model.print_params()):
      param.tofile('/content/gdrive/My Drive/EMNIST_model_saving_file'+str(k))
    print('iter {:d}: model parameters have been saved'.format(i+1))

----------iter: 1/50, loss: 5.347049----------
----------iter: 2/50, loss: 5.394904----------
----------iter: 3/50, loss: 5.237479----------
----------iter: 4/50, loss: 5.069438----------
----------iter: 5/50, loss: 6.503512----------
iter 5: model parameters have been saved
----------iter: 6/50, loss: 5.025169----------
----------iter: 7/50, loss: 5.151392----------
----------iter: 8/50, loss: 5.139250----------
----------iter: 9/50, loss: 5.315397----------
----------iter: 10/50, loss: 5.196726----------
iter 10: model parameters have been saved
----------iter: 11/50, loss: 5.296342----------
----------iter: 12/50, loss: 5.419592----------
----------iter: 13/50, loss: 5.246149----------
----------iter: 14/50, loss: 5.174092----------
----------iter: 15/50, loss: 5.090343----------
iter 15: model parameters have been saved
----------iter: 16/50, loss: 5.110538----------
----------iter: 17/50, loss: 4.886448----------
----------iter: 18/50, loss: 5.066410----------
----------iter: 19/5

AFL model

In [ ]:
learning_rate = 1
lambda_learning_rate = 1e-5
CNN_AFL_fit = AFL(emnist_train, IDs, my_CNN, initial_params, learning_rate, lambda_learning_rate)

In [ ]:
CNN_AFL_fit.lambdas = lambdas

In [ ]:
ITER = 1000
for i in range(ITER):
  loss = CNN_AFL_fit.train(BATCH_SIZE)
  print('----------iter: {:d}/{:d}, loss: {:f}----------'.format(i+1, ITER, loss))
  if (i+1) % 100 == 0:
    final_model = CNN_AFL_fit.model
    for k, param in enumerate(final_model.print_params()):
      param.tofile('/content/gdrive/My Drive/AFL_EMNIST_model_saving_file'+str(k))
    final_lambdas = np.array(CNN_AFL_fit.lambdas)
    final_lambdas.tofile('/content/gdrive/My Drive/AFL_EMNIST_model_saving_file'+str(6))
    print('iter {:d}: model parameters have been saved'.format(i+1))

----------iter: 1/1000, loss: 5388.071660----------
----------iter: 2/1000, loss: 5414.745753----------
----------iter: 3/1000, loss: 5387.443527----------
----------iter: 4/1000, loss: 5460.158915----------
----------iter: 5/1000, loss: 5413.954636----------
----------iter: 6/1000, loss: 5435.473963----------
----------iter: 7/1000, loss: 5393.864721----------
----------iter: 8/1000, loss: 5486.169899----------
----------iter: 9/1000, loss: 5402.587311----------
----------iter: 10/1000, loss: 5410.771725----------
----------iter: 11/1000, loss: 5399.324824----------
----------iter: 12/1000, loss: 5420.729914----------
----------iter: 13/1000, loss: 5390.795712----------
----------iter: 14/1000, loss: 5482.818870----------
----------iter: 15/1000, loss: 5398.002553----------
----------iter: 16/1000, loss: 5414.885020----------
----------iter: 17/1000, loss: 5374.869018----------
----------iter: 18/1000, loss: 5389.519623----------
----------iter: 19/1000, loss: 5375.671496----------
--

## Test Example

define test datasets

In [ ]:
emnist_test_data = [Client_Data(emnist_test, id) for id in IDs]

### FedAvg model results

In [ ]:
final_model = CNN_AVGM_fit.model

nums1 = []
accs1 = []
for i, id in enumerate(IDs):
  client_data = emnist_test_data[i]
  accuracy = final_model.predict_error(client_data)
  nums1.append(len(client_data))
  accs1.append(accuracy)
  print('client: {:s} accuracy: {:f}'.format(id, accuracy))

client: f0000_14 accuracy: 1.000000
client: f0001_41 accuracy: 1.000000
client: f0005_26 accuracy: 1.000000
client: f0006_12 accuracy: 0.916667
client: f0008_45 accuracy: 1.000000
client: f0011_13 accuracy: 0.916667
client: f0014_19 accuracy: 1.000000
client: f0016_39 accuracy: 1.000000
client: f0017_07 accuracy: 1.000000
client: f0022_10 accuracy: 0.818182
client: f0023_08 accuracy: 0.923077
client: f0029_05 accuracy: 1.000000
client: f0030_12 accuracy: 1.000000
client: f0031_02 accuracy: 1.000000
client: f0032_27 accuracy: 1.000000
client: f0034_08 accuracy: 1.000000
client: f0035_19 accuracy: 0.923077
client: f0036_39 accuracy: 1.000000
client: f0037_16 accuracy: 1.000000
client: f0038_42 accuracy: 0.888889
client: f0039_14 accuracy: 1.000000
client: f0040_25 accuracy: 1.000000
client: f0041_30 accuracy: 0.818182
client: f0042_43 accuracy: 1.000000
client: f0043_10 accuracy: 0.916667
client: f0048_00 accuracy: 0.857143
client: f0049_32 accuracy: 1.000000
client: f0052_42 accuracy: 1

average prediction accuracy

In [ ]:
np.sum(np.multiply(accs1, nums1)) / np.sum(nums1)

0.9340468260188087

worst performance

In [ ]:
min(accs1)

0.25

highschool

In [ ]:
int_ids = np.array([int(id.split('_')[0][1:]) for id in IDs])

In [ ]:
highschool_ids = np.array(IDs)[[i for i, a in enumerate(int_ids) if a >= 2100 and a <= 2599]].tolist()
highschool_test_data = [Client_Data(emnist_test, id) for id in highschool_ids]
nums1 = []
accs1 = []
for i, id in enumerate(highschool_ids):
  client_data = highschool_test_data[i]
  accuracy = final_model.predict_error(client_data)
  nums1.append(len(client_data))
  accs1.append(accuracy)
  print('client: {:s} accuracy: {:f}'.format(id, accuracy))

client: f2100_97 accuracy: 1.000000
client: f2101_62 accuracy: 0.750000
client: f2102_52 accuracy: 1.000000
client: f2103_60 accuracy: 1.000000
client: f2104_50 accuracy: 1.000000
client: f2105_51 accuracy: 0.857143
client: f2106_66 accuracy: 0.857143
client: f2107_53 accuracy: 0.923077
client: f2108_54 accuracy: 0.909091
client: f2109_65 accuracy: 0.833333
client: f2110_56 accuracy: 1.000000
client: f2111_87 accuracy: 0.538462
client: f2112_65 accuracy: 0.928571
client: f2113_94 accuracy: 0.785714
client: f2114_50 accuracy: 0.769231
client: f2115_77 accuracy: 0.833333
client: f2116_72 accuracy: 0.923077
client: f2117_54 accuracy: 1.000000
client: f2118_60 accuracy: 0.923077
client: f2119_56 accuracy: 1.000000
client: f2120_61 accuracy: 0.909091
client: f2121_67 accuracy: 1.000000
client: f2122_65 accuracy: 1.000000
client: f2123_63 accuracy: 1.000000
client: f2124_66 accuracy: 0.636364
client: f2125_64 accuracy: 0.846154
client: f2126_57 accuracy: 0.750000
client: f2127_59 accuracy: 0

In [ ]:
np.sum(np.multiply(accs1, nums1)) / np.sum(nums1)

0.8751398880895284

census

In [ ]:
census_ids = np.array(IDs)[[i for i, a in enumerate(int_ids) if a < 2100 or a > 2599]].tolist()
census_test_data = [Client_Data(emnist_test, id) for id in census_ids]
nums1 = []
accs1 = []
for i, id in enumerate(census_ids):
  client_data = census_test_data[i]
  accuracy = final_model.predict_error(client_data)
  nums1.append(len(client_data))
  accs1.append(accuracy)
  print('client: {:s} accuracy: {:f}'.format(id, accuracy))

client: f0000_14 accuracy: 1.000000
client: f0001_41 accuracy: 1.000000
client: f0005_26 accuracy: 1.000000
client: f0006_12 accuracy: 0.916667
client: f0008_45 accuracy: 1.000000
client: f0011_13 accuracy: 0.916667
client: f0014_19 accuracy: 1.000000
client: f0016_39 accuracy: 1.000000
client: f0017_07 accuracy: 1.000000
client: f0022_10 accuracy: 0.818182
client: f0023_08 accuracy: 0.923077
client: f0029_05 accuracy: 1.000000
client: f0030_12 accuracy: 1.000000
client: f0031_02 accuracy: 1.000000
client: f0032_27 accuracy: 1.000000
client: f0034_08 accuracy: 1.000000
client: f0035_19 accuracy: 0.923077
client: f0036_39 accuracy: 1.000000
client: f0037_16 accuracy: 1.000000
client: f0038_42 accuracy: 0.888889
client: f0039_14 accuracy: 1.000000
client: f0040_25 accuracy: 1.000000
client: f0041_30 accuracy: 0.818182
client: f0042_43 accuracy: 1.000000
client: f0043_10 accuracy: 0.916667
client: f0048_00 accuracy: 0.857143
client: f0049_32 accuracy: 1.000000
client: f0052_42 accuracy: 1

In [ ]:
np.sum(np.multiply(accs1, nums1)) / np.sum(nums1)

0.94470312635567

fairness

In [ ]:
np.std(accs1)

0.10486110181117585

In [ ]:
np.mean(accs1) / np.sqrt(np.mean(np.square(accs1)))

0.9935991830365997

In [ ]:
-np.sum([acc / np.sum(accs1) * np.math.log(acc / np.sum(accs1)) for acc in accs1])

8.119425506468655

### AFL model results

In [ ]:
final_model = CNN_AFL_fit.model

accs2 = []
nums2 = []
for i, id in enumerate(IDs):
  client_data = emnist_test_data[i]
  accuracy = final_model.predict_error(client_data)
  accs2.append(accuracy)
  nums2.append(len(client_data))
  print('client: {:s} accuracy: {:f}'.format(id, accuracy))

client: f0000_14 accuracy: 0.454545
client: f0001_41 accuracy: 0.230769
client: f0005_26 accuracy: 0.666667
client: f0006_12 accuracy: 0.666667
client: f0008_45 accuracy: 0.500000
client: f0011_13 accuracy: 0.500000
client: f0014_19 accuracy: 0.615385
client: f0016_39 accuracy: 0.400000
client: f0017_07 accuracy: 0.461538
client: f0022_10 accuracy: 0.636364
client: f0023_08 accuracy: 0.461538
client: f0029_05 accuracy: 0.250000
client: f0030_12 accuracy: 0.333333
client: f0031_02 accuracy: 0.454545
client: f0032_27 accuracy: 0.666667
client: f0034_08 accuracy: 0.090909
client: f0035_19 accuracy: 0.384615
client: f0036_39 accuracy: 0.545455
client: f0037_16 accuracy: 0.357143
client: f0038_42 accuracy: 0.555556
client: f0039_14 accuracy: 0.545455
client: f0040_25 accuracy: 0.461538
client: f0041_30 accuracy: 0.272727
client: f0042_43 accuracy: 0.444444
client: f0043_10 accuracy: 0.333333
client: f0048_00 accuracy: 0.571429
client: f0049_32 accuracy: 0.307692
client: f0052_42 accuracy: 0

In [ ]:
CNN_AFL_fit.lambdas.tolist()

[0.0,
 0.0,
 0.0034207204947177433,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.5269845456261075e-05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004111612015694762,
 0.0,
 0.0020036172589961445,
 0.0,
 0.0006152376732531962,
 0.0,
 0.0,
 0.0,
 4.439319345392819e-05,
 0.0,
 0.0,
 0.001128392986983437,
 0.0,
 0.0010765871653262548,
 0.0,
 0.0,
 0.0003479276082221447,
 0.0,
 0.0,
 0.00318746679279246,
 0.0,
 0.0012323930511180367,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00010567825171389236,
 0.0,
 0.0,
 0.0,
 0.0003057880375091014,
 0.0,
 0.0010239629385177102,
 0.0,
 0.0,
 0.001841407694548744,
 0.009721364463299891,
 0.0,
 0.0,
 0.001787829702824739,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0002015155865608033,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00018986314031519536,
 0.0,
 0

In [ ]:
np.sum(np.multiply(accs2, nums2)) / np.sum(nums2)

0.4406103056426332

In [ ]:
min(accs2)

0.0

In [ ]:
np.std(accs2)

0.14326745903814683

In [ ]:
np.mean(accs2) / np.sqrt(np.mean(np.square(accs2)))

0.9958422929182217

In [ ]:
-np.sum([acc / np.sum(accs2) * np.math.log(acc / np.sum(accs2)) for acc in accs2])

2.991241285885079